In [6]:
using DataFrames
using CSV
using Cbc
include("../instances_io.jl")
include("PL.jl");

L = [3] # number of lines in the grid-like graph
C = [3,4,5,6,7,8] # number of columns in the grid-like graph
K = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20] # number of edges the adversary can penalize
MAXC = 5 # range of initial edge cost
MAXD = [0,1,2,3,4,5,6,7,8,9,10,20,30,40,50,75,100] # range of penalized edge cost
l = 5 # number of lines in the grid-like graph
c = 5 # number of columns in the grid-like graph
k = 10 # number of edges the adversary can penalize
maxc = 5 # range of initial edge cost
maxd = 50 # range of penalized edge cost

df = DataFrame(Param = String[], Param_value = Int[], Iteration=Int[], Time=Float64[], Cuts=Int[])
for l in L
    for it in 1:2
        sv = generate(l, c, k, maxc, maxd);
        timer = @elapsed obj, x_opt, count = solve_PL2(sv, CbcSolver())
        push!(df, ("l", l, it, timer, count))
    end
end
CSV.write("dataframe.csv", df)
        

10.0 <= value <= 1.0e8
13.0 <= value <= 138.0
19.0 <= value <= 138.0
43.0 <= value <= 116.0
18.0 <= value <= 98.0
44.0 <= value <= 95.0
48.0 <= value <= 87.0
48.0 <= value <= 87.0
57.0 <= value <= 76.0
44.0 <= value <= 76.0
47.0 <= value <= 76.0
49.0 <= value <= 76.0
61.0 <= value <= 75.0
64.0 <= value <= 67.0
67.0 <= value <= 67.0
13.0 <= value <= 1.0e8
14.0 <= value <= 198.0
16.0 <= value <= 145.0
52.0 <= value <= 135.0
19.0 <= value <= 104.0
19.0 <= value <= 92.0
51.0 <= value <= 92.0
49.0 <= value <= 87.0
48.0 <= value <= 87.0
18.0 <= value <= 81.0
51.0 <= value <= 80.0
50.0 <= value <= 77.0
51.0 <= value <= 77.0
56.0 <= value <= 76.0
51.0 <= value <= 74.0
53.0 <= value <= 74.0
51.0 <= value <= 74.0
22.0 <= value <= 74.0
54.0 <= value <= 70.0
19.0 <= value <= 69.0
19.0 <= value <= 67.0
54.0 <= value <= 65.0
50.0 <= value <= 65.0
18.0 <= value <= 65.0
52.0 <= value <= 64.0
54.0 <= value <= 64.0
52.0 <= value <= 64.0
54.0 <= value <= 64.0
52.0 <= value <= 64.0
54.0 <= value <= 64.0
5

"dataframe.csv"